## Importación de librerías

In [1]:
import pandas as pd
import os
import glob
import json
import numpy as np
import math

## Configuración de variables globales

In [7]:
with open('setup.json', 'r') as f:
    SETUP_JSON = json.load(f)
DATASETS_PATH = SETUP_JSON['datasets_path'] # Ruta de la carpeta de los datasets,
DATASETS_FOLDER = os.path.join(os.getcwd(), DATASETS_PATH) # Carpeta de los datasets,
DATASETS = glob.glob(os.path.join(DATASETS_FOLDER, '*.csv')) # Lista de los datasets
OUTPUT_CSV = SETUP_JSON['dataset_csv'] # Fichero CSV de salida
OUTPUT_PARQUET = SETUP_JSON['dataset_parquet'] # Fichero Parquet de salida
TAKE_FULL_DATASET = SETUP_JSON['take_full_dataset'] # Obtener el dataset completo o una muestra
NA_VAL = SETUP_JSON['navalues'] # Valores a considerar como NaN
balances = SETUP_JSON['balances'] # Balances a considerar

## Análisis de los datasets

Obtener todos los encabezados diferentes

In [3]:
unique_headers = set()
for dataset in DATASETS:
    df_dtypes = pd.read_csv(dataset, nrows=1).dtypes.keys()
    unique_headers.add(tuple(df_dtypes))
for header in unique_headers:
    print(header)

('Flow ID', 'Src IP', 'Src Port', 'Dst IP', 'Dst Port', 'Protocol', 'Timestamp', 'Flow Duration', 'Tot Fwd Pkts', 'Tot Bwd Pkts', 'TotLen Fwd Pkts', 'TotLen Bwd Pkts', 'Fwd Pkt Len Max', 'Fwd Pkt Len Min', 'Fwd Pkt Len Mean', 'Fwd Pkt Len Std', 'Bwd Pkt Len Max', 'Bwd Pkt Len Min', 'Bwd Pkt Len Mean', 'Bwd Pkt Len Std', 'Flow Byts/s', 'Flow Pkts/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Tot', 'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Tot', 'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags', 'Bwd PSH Flags', 'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Len', 'Bwd Header Len', 'Fwd Pkts/s', 'Bwd Pkts/s', 'Pkt Len Min', 'Pkt Len Max', 'Pkt Len Mean', 'Pkt Len Std', 'Pkt Len Var', 'FIN Flag Cnt', 'SYN Flag Cnt', 'RST Flag Cnt', 'PSH Flag Cnt', 'ACK Flag Cnt', 'URG Flag Cnt', 'CWE Flag Count', 'ECE Flag Cnt', 'Down/Up Ratio', 'Pkt Size Avg', 'Fwd Seg Size Avg', 'Bwd Seg Size Avg', 'Fwd Byts/b Avg', 'F

Separar los grupos de datasets por encabezados diferentes

In [4]:
datasets_by_header = {}
for header in unique_headers:
    datasets_group = []
    for dataset in DATASETS:
        df_dtypes = pd.read_csv(dataset, nrows=1).dtypes.keys()
        if tuple(df_dtypes) == header:
            datasets_group.append(dataset)
    datasets_by_header[header] = datasets_group
    print(f"Grupo de datasets:")
    for dataset in datasets_group:
        print(dataset)
    print(f"Número de columnas del grupo de datasets: {len(header)}")
    print()

Grupo de datasets:
c:\Users\isard\Desktop\AI-for-Traffic-network-classify\datasets\02-20-2018.csv
Número de columnas del grupo de datasets: 84

Grupo de datasets:
c:\Users\isard\Desktop\AI-for-Traffic-network-classify\datasets\02-14-2018.csv
c:\Users\isard\Desktop\AI-for-Traffic-network-classify\datasets\02-15-2018.csv
c:\Users\isard\Desktop\AI-for-Traffic-network-classify\datasets\02-16-2018.csv
c:\Users\isard\Desktop\AI-for-Traffic-network-classify\datasets\02-21-2018.csv
c:\Users\isard\Desktop\AI-for-Traffic-network-classify\datasets\02-22-2018.csv
c:\Users\isard\Desktop\AI-for-Traffic-network-classify\datasets\02-23-2018.csv
c:\Users\isard\Desktop\AI-for-Traffic-network-classify\datasets\02-28-2018.csv
c:\Users\isard\Desktop\AI-for-Traffic-network-classify\datasets\03-01-2018.csv
c:\Users\isard\Desktop\AI-for-Traffic-network-classify\datasets\03-02-2018.csv
Número de columnas del grupo de datasets: 80



Obtener etiquetas diferentes encontradas en cada grupo de datasets

In [5]:
datasets_by_num_labels = {}

for datasets_group in datasets_by_header.values():
    unique_labels = set()
    for dataset in datasets_group:
        all_labels = list(pd.read_csv(dataset, usecols=["Label"])["Label"])
        for label in all_labels:
            unique_labels.add(label)
    datasets_by_num_labels[tuple(datasets_group)] = len(unique_labels)
    print(f"Grupo de datasets:")
    for dataset in datasets_group:
        print(dataset)
    print(f"Etiquetas diferentes encontradas: ", unique_labels)
    print()


Grupo de datasets:
c:\Users\isard\Desktop\AI-for-Traffic-network-classify\datasets\02-20-2018.csv
Etiquetas diferentes encontradas:  {'Benign', 'DDoS attacks-LOIC-HTTP'}
Grupo de datasets:
c:\Users\isard\Desktop\AI-for-Traffic-network-classify\datasets\02-14-2018.csv
c:\Users\isard\Desktop\AI-for-Traffic-network-classify\datasets\02-15-2018.csv
c:\Users\isard\Desktop\AI-for-Traffic-network-classify\datasets\02-16-2018.csv
c:\Users\isard\Desktop\AI-for-Traffic-network-classify\datasets\02-21-2018.csv
c:\Users\isard\Desktop\AI-for-Traffic-network-classify\datasets\02-22-2018.csv
c:\Users\isard\Desktop\AI-for-Traffic-network-classify\datasets\02-23-2018.csv
c:\Users\isard\Desktop\AI-for-Traffic-network-classify\datasets\02-28-2018.csv
c:\Users\isard\Desktop\AI-for-Traffic-network-classify\datasets\03-01-2018.csv
c:\Users\isard\Desktop\AI-for-Traffic-network-classify\datasets\03-02-2018.csv
Etiquetas diferentes encontradas:  {'DDOS attack-LOIC-UDP', 'SSH-Bruteforce', 'Label', 'DoS attacks-

Seleccionar el grupo de datasets que abarca más etiquetas diferentes

In [6]:
for datasets_group, num_labels in datasets_by_num_labels.items():
    if (num_labels) == max(datasets_by_num_labels.values()):
        datasets_selected = datasets_group
print(f"Selección de datasets a procesar:")
for dataset in datasets_selected:
    print(dataset)

Selección de datasets a procesar:
c:\Users\isard\Desktop\AI-for-Traffic-network-classify\datasets\02-14-2018.csv
c:\Users\isard\Desktop\AI-for-Traffic-network-classify\datasets\02-15-2018.csv
c:\Users\isard\Desktop\AI-for-Traffic-network-classify\datasets\02-16-2018.csv
c:\Users\isard\Desktop\AI-for-Traffic-network-classify\datasets\02-21-2018.csv
c:\Users\isard\Desktop\AI-for-Traffic-network-classify\datasets\02-22-2018.csv
c:\Users\isard\Desktop\AI-for-Traffic-network-classify\datasets\02-23-2018.csv
c:\Users\isard\Desktop\AI-for-Traffic-network-classify\datasets\02-28-2018.csv
c:\Users\isard\Desktop\AI-for-Traffic-network-classify\datasets\03-01-2018.csv
c:\Users\isard\Desktop\AI-for-Traffic-network-classify\datasets\03-02-2018.csv


## Combinar datasets

Unimos todos los datasets seleccionados en un sólo dataframe

In [35]:
df = pd.DataFrame()
if TAKE_FULL_DATASET:
    for dataset in datasets_selected:
        current_df = pd.read_csv(dataset, sep=',', low_memory=False, na_values=NA_VAL) 
        df = pd.concat([df, current_df], ignore_index=True)
        print(f"Dataset concatenado: {dataset}, Dimensiones: {current_df.shape}")

else:
    for dataset in datasets_selected:
        current_df = pd.read_csv(dataset, sep=',', low_memory=False, na_values=NA_VAL)
        
        # Agregar una columna auxiliar que cuenta la aparición de cada Label
        current_df["Label_Count"] = current_df.groupby("Label").cumcount() + 1

        # Filtrar solo las primeras N apariciones de cada Label
        copy_df = current_df[current_df.apply(lambda row: row["Label_Count"] <= balances.get(row["Label"], 0), axis=1)]

        # Eliminar columna auxiliar antes de concatenar
        copy_df.drop(columns=["Label_Count"], inplace=True)

        df = pd.concat([df, copy_df], ignore_index=True)

        print(f"Dataset concatenado: {dataset}, Dimensiones: {copy_df.shape}")


Dataset concatenado: c:\Users\isard\Desktop\AI-for-Traffic-network-classify\datasets\02-14-2018.csv, Dimensiones: (1048575, 80)
Dataset concatenado: c:\Users\isard\Desktop\AI-for-Traffic-network-classify\datasets\02-15-2018.csv, Dimensiones: (1048575, 80)
Dataset concatenado: c:\Users\isard\Desktop\AI-for-Traffic-network-classify\datasets\02-16-2018.csv, Dimensiones: (1048575, 80)
Dataset concatenado: c:\Users\isard\Desktop\AI-for-Traffic-network-classify\datasets\02-21-2018.csv, Dimensiones: (1048575, 80)
Dataset concatenado: c:\Users\isard\Desktop\AI-for-Traffic-network-classify\datasets\02-22-2018.csv, Dimensiones: (1048575, 80)
Dataset concatenado: c:\Users\isard\Desktop\AI-for-Traffic-network-classify\datasets\02-23-2018.csv, Dimensiones: (1048575, 80)
Dataset concatenado: c:\Users\isard\Desktop\AI-for-Traffic-network-classify\datasets\02-28-2018.csv, Dimensiones: (613104, 80)
Dataset concatenado: c:\Users\isard\Desktop\AI-for-Traffic-network-classify\datasets\03-01-2018.csv, Dime

In [36]:
df.shape

(8284254, 80)

## Análisis

In [37]:
# Verificar si hay encabezados duplicados
df_headers = df[df['Label'] == 'Label']
df_headers.shape[0]

59

In [38]:
# Ver la cantidad de filas con valores faltantes
df.isna().any(axis=1).value_counts()

False    8261300
True       22954
Name: count, dtype: int64

In [40]:
# Ver la cantidad de filas por etiqueta
df['Label'].value_counts()

Label
Benign                      6112151
DDOS attack-HOIC             686012
DoS attacks-Hulk             461912
Bot                          286191
FTP-BruteForce               193360
SSH-Bruteforce               187589
Infilteration                161934
DoS attacks-SlowHTTPTest     139890
DoS attacks-GoldenEye         41508
DoS attacks-Slowloris         10990
DDOS attack-LOIC-UDP           1730
Brute Force -Web                611
Brute Force -XSS                230
SQL Injection                    87
Label                            59
Name: count, dtype: int64

In [41]:
# Ver los tipos de datos de las columnas
df.dtypes

Dst Port         object
Protocol         object
Timestamp        object
Flow Duration    object
Tot Fwd Pkts     object
                  ...  
Idle Mean        object
Idle Std         object
Idle Max         object
Idle Min         object
Label            object
Length: 80, dtype: object

## Limpieza

In [42]:
# Eliminar encabezados repetidos
df = df[df['Label'] != 'Label']
df.shape

(8284195, 80)

In [44]:
# Eliminar los valores faltantes
df = df.dropna()
df.shape

(8261241, 80)

## Normalización

Asignar número a cada tipo de etiqueta

In [ ]:
label_mapping = {label: int(i) for i, label in enumerate(unique_labels)}

In [56]:
df['Label Code'] = df['Label'].map(label_mapping)
df['Label Code'].value_counts()

Label Code
0     6077145
8      686012
6      461912
13     286191
1      193354
2      187589
12     160639
5      139890
3       41508
4       10990
7        1730
9         611
10        230
11         87
Name: count, dtype: int64

Cambiar tipo de cada columna a su tipo correspondiente

In [47]:
# Asignar formato decimal a columnas numéricas
for col, dtype in df.dtypes.to_dict().items():
    temp_col = df[col].dropna()
    numeric_col = pd.to_numeric(temp_col, errors='coerce').notna().all()
    if numeric_col:
        df[col] = pd.to_numeric(temp_col, errors='coerce').astype('float64')
        
# Asignar formato entero a columnas que no contienen decimales
df["Protocol"] = df["Protocol"].astype(int)
df["Dst Port"] = df["Dst Port"].astype(int)
df["Label Code"] = df["Label Code"].astype(int)

# Conversión de formato de timestamp a formato legible para el modelo
df['Timestamp'] = pd.to_datetime(df['Timestamp'], format='%d/%m/%Y %H:%M:%S', errors='coerce') # Convertimos la timestamp a formato fecha de pandas
df['Timestamp'] = np.sin(2*math.pi*df['Timestamp'].dt.hour/24)  # Convertimos la fecha en un ciclo legible para el modelo

In [48]:
df.dtypes

Dst Port           int64
Protocol           int64
Timestamp        float64
Flow Duration    float64
Tot Fwd Pkts     float64
                  ...   
Idle Std         float64
Idle Max         float64
Idle Min         float64
Label             object
Label Code         int64
Length: 81, dtype: object

In [49]:
# Ver el número de filas con valores infinitos
X_df = df.drop(columns=['Label', 'Label Code'])
cols_with_inf = np.isinf(X_df).any(axis=1)
cols_with_inf.value_counts()

False    8247888
True       13353
Name: count, dtype: int64

In [50]:
# Eliminar filas con valores infinitos
df = df[~df.isin([np.inf, -np.inf]).any(axis=1)]
df.shape

(8247888, 81)

In [51]:
df.reset_index(drop=True, inplace=True)

In [52]:
df.head()

,Dst Port,Protocol,Timestamp,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,...,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label,Label Code
0,0,0,0.866025,112641719.0,3.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,56320859.5,139.300036,56320958.0,56320761.0,Benign,0
1,0,0,0.866025,112641466.0,3.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,56320733.0,114.551299,56320814.0,56320652.0,Benign,0
2,0,0,0.866025,112638623.0,3.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,56319311.5,301.934596,56319525.0,56319098.0,Benign,0
3,22,6,0.866025,6453966.0,15.0,10.0,1239.0,2273.0,744.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,Benign,0
4,22,6,0.866025,8804066.0,14.0,11.0,1143.0,2209.0,744.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,Benign,0


In [53]:
df.tail()

,Dst Port,Protocol,Timestamp,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,...,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label,Label Code
8247883,3389,6,0.5,3982183.0,14.0,8.0,1442.0,1731.0,725.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign,0
8247884,3389,6,0.5,3802316.0,14.0,8.0,1440.0,1731.0,725.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign,0
8247885,3389,6,0.5,4004239.0,14.0,8.0,1459.0,1731.0,741.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign,0
8247886,3389,6,0.5,3998435.0,14.0,8.0,1459.0,1731.0,741.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign,0
8247887,3389,6,0.5,3972651.0,14.0,8.0,1439.0,1731.0,725.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign,0


## Exportación del dataframe final en formato Parquet

In [55]:
# Guardar en Parquet
df.to_parquet(OUTPUT_PARQUET, index=False)
print(f"Archivo parquet guardado como {OUTPUT_PARQUET}")

Archivo parquet guardado como clean_dataset.parquet
